In [2]:
import nltk #need for dealing with text
import os # need for looping through folders
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math # need for computing TF-IDF score

In [3]:
pip install num2words

Note: you may need to restart the kernel to use updated packages.


In [4]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words

In [5]:
title = "used_files"
os.chdir("C:/mini_newsgroups/comp.graphics/")
paths= []
for (dirpath, dirnames, filenames) in os.walk(str(os.getcwd())+'/'+title+'/'): 
    for i in filenames: 
        paths.append(str(dirpath)+str("\\")+i)
        
print(dirpath)
paths[0]
print(len(paths))

C:\mini_newsgroups\comp.graphics/used_files/
4


In [6]:
myfile = open("C:/mini_newsgroups/comp.graphics/used_files/37916")
txt = myfile.read()
print(txt)
myfile.close()

Path: cantaloupe.srv.cs.cmu.edu!crabapple.srv.cs.cmu.edu!fs7.ece.cmu.edu!europa.eng.gtefsd.com!gatech!asuvax!cs.utexas.edu!zaphod.mps.ohio-state.edu!saimiri.primate.wisc.edu!usenet.coe.montana.edu!news.u.washington.edu!uw-beaver!cs.ubc.ca!unixg.ubc.ca!kakwa.ucs.ualberta.ca!ersys!joth
From: joth@ersys.edmonton.ab.ca (Joe Tham)
Newsgroups: comp.graphics
Subject: Where can I find SIPP?
Message-ID: <yFXJ2B2w165w@ersys.edmonton.ab.ca>
Date: Mon, 05 Apr 93 14:58:21 MDT
Organization: Edmonton Remote Systems #2, Edmonton, AB, Canada
Lines: 11

        I recently got a file describing a library of rendering routines 
called SIPP (SImple Polygon Processor).  Could anyone tell me where I can 
FTP the source code and which is the newest version around?
        Also, I've never used Renderman so I was wondering if Renderman 
is like SIPP?  ie. a library of rendering routines which one uses to make 
a program that creates the image...

                                        Thanks,  Joe Tham

--
Jo

# Exercise 1: Explore content of files. Write your code below.

In [7]:
def print_doc(id):
    file = open(dirpath+'/'+ filenames[id],'r', encoding='cp1250')
    text = file.read().strip()
    file.close()
    print(text)
    
print_doc(0)
print_doc(1)

Path: cantaloupe.srv.cs.cmu.edu!crabapple.srv.cs.cmu.edu!fs7.ece.cmu.edu!europa.eng.gtefsd.com!gatech!asuvax!cs.utexas.edu!zaphod.mps.ohio-state.edu!saimiri.primate.wisc.edu!usenet.coe.montana.edu!news.u.washington.edu!uw-beaver!cs.ubc.ca!unixg.ubc.ca!kakwa.ucs.ualberta.ca!ersys!joth
From: joth@ersys.edmonton.ab.ca (Joe Tham)
Newsgroups: comp.graphics
Subject: Where can I find SIPP?
Message-ID: <yFXJ2B2w165w@ersys.edmonton.ab.ca>
Date: Mon, 05 Apr 93 14:58:21 MDT
Organization: Edmonton Remote Systems #2, Edmonton, AB, Canada
Lines: 11

        I recently got a file describing a library of rendering routines 
called SIPP (SImple Polygon Processor).  Could anyone tell me where I can 
FTP the source code and which is the newest version around?
        Also, I've never used Renderman so I was wondering if Renderman 
is like SIPP?  ie. a library of rendering routines which one uses to make 
a program that creates the image...

                                        Thanks,  Joe Tham

--
Jo

# Exercise 2: Do the necessary data preparation (Similar to what did you do in the lab 06, You also need to remove stop words)

In [8]:
def remove_single_characters(data):
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if len(w) > 1:
            new_text = new_text + " " + w
    return np.char.strip(new_text)

def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

def remove_header(data):
    try:
        ind = data.index('\n\n')
        data = data[ind:]
    except:
        print("No Header")
    return data

def convert_numbers(data):
    data = np.char.replace(data, "0", " zero ")
    data = np.char.replace(data, "1", " one ")
    data = np.char.replace(data, "2", " two ")
    data = np.char.replace(data, "3", " three ")
    data = np.char.replace(data, "4", " four ")
    data = np.char.replace(data, "5", " five ")
    data = np.char.replace(data, "6", " six ")
    data = np.char.replace(data, "7", " seven ")
    data = np.char.replace(data, "8", " eight ")
    data = np.char.replace(data, "9", " nine ")
    return data

def stemming(data):
    stemmer= PorterStemmer()
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return np.char.strip(new_text)

def convert_lower_case(data):
    return np.char.lower(data)

def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], '')
        data = np.char.replace(data, " ", " ")
    data = np.char.replace(data, ',', '')
    return data

def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words:
            new_text = new_text + " " + w
    return np.char.strip(new_text)

In [9]:
def preprocess(data):
    data = remove_header(data)
    data = remove_single_characters(data)
    data = remove_apostrophe(data)
    data = convert_numbers(data)
    data = stemming(data)
    data = convert_lower_case(data)
    data = remove_punctuation(data)
    data = remove_stop_words(data)
    return data

In [10]:
processed_text = []

for i in range(len(filenames)):
    file = open(dirpath+'/'+ filenames[i], 'r', encoding='cp1250',errors='ignore')
    text = file.read().strip()
    file.close()
    
    processed_text.append(word_tokenize(str(preprocess(text))))
    print(processed_text)

[['recent', 'got', 'file', 'describ', 'librari', 'render', 'routin', 'call', 'sipp', 'simpl', 'polygon', 'processor', 'could', 'anyon', 'tell', 'ftp', 'sourc', 'code', 'newest', 'version', 'around', 'also', 'never', 'use', 'renderman', 'wa', 'wonder', 'renderman', 'like', 'sipp', 'ie', 'librari', 'render', 'routin', 'one', 'use', 'make', 'program', 'creat', 'imag', 'thank', 'joe', 'tham', 'joe', 'tham', 'joth', 'ersysedmontonabca']]
[['recent', 'got', 'file', 'describ', 'librari', 'render', 'routin', 'call', 'sipp', 'simpl', 'polygon', 'processor', 'could', 'anyon', 'tell', 'ftp', 'sourc', 'code', 'newest', 'version', 'around', 'also', 'never', 'use', 'renderman', 'wa', 'wonder', 'renderman', 'like', 'sipp', 'ie', 'librari', 'render', 'routin', 'one', 'use', 'make', 'program', 'creat', 'imag', 'thank', 'joe', 'tham', 'joe', 'tham', 'joth', 'ersysedmontonabca'], ['hello', 'everybodi', 'use', 'pixar', 'renderman', 'three', 'scene', 'descript', 'languag', 'creat', 'three', 'world', 'pleas

# Exercise 3: Calculating DF for all words found in processed_text list. Print the first five inputs
'recent': 1,
 'got': 1,
 'file': 1,
 'describ': 1,
 'librari': 2,


In [11]:
DF = {}
N= len(processed_text)

for i in range(N):
    tokens = processed_text[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w]= {i}
            
for i in DF:
    DF[i] = len(DF[i])

In [12]:
DF

{'recent': 1,
 'got': 1,
 'file': 1,
 'describ': 1,
 'librari': 2,
 'render': 2,
 'routin': 1,
 'call': 1,
 'sipp': 1,
 'simpl': 2,
 'polygon': 1,
 'processor': 1,
 'could': 2,
 'anyon': 2,
 'tell': 1,
 'ftp': 2,
 'sourc': 1,
 'code': 1,
 'newest': 1,
 'version': 2,
 'around': 1,
 'also': 2,
 'never': 1,
 'use': 2,
 'renderman': 2,
 'wa': 2,
 'wonder': 1,
 'like': 2,
 'ie': 1,
 'one': 2,
 'make': 1,
 'program': 1,
 'creat': 2,
 'imag': 2,
 'thank': 2,
 'joe': 1,
 'tham': 1,
 'joth': 1,
 'ersysedmontonabca': 1,
 'hello': 1,
 'everybodi': 1,
 'pixar': 1,
 'three': 1,
 'scene': 1,
 'descript': 1,
 'languag': 1,
 'world': 1,
 'pleas': 1,
 'help': 1,
 'next': 1,
 'document': 1,
 'nextstep': 1,
 'avail': 2,
 'veri': 2,
 'complic': 1,
 'surfac': 1,
 'shader': 1,
 'bring': 1,
 'life': 1,
 'appli': 1,
 'shadow': 1,
 'reflect': 1,
 'far': 1,
 'understand': 1,
 'defin': 1,
 'environment': 1,
 'map': 1,
 'produc': 1,
 'know': 3,
 'ani': 2,
 'advis': 1,
 'rib': 1,
 'exampl': 1,
 'appreci': 1,
 'adv

In [20]:
total_vocab =[x for x in DF]
print(total_vocab[:5])

['recent', 'got', 'file', 'describ', 'librari']


In [ ]:
#total_vocab =[]
#for x in DF:
#    total_vocab.append(x)

# Exercise 4: What is the total vocabulary size? 144

In [13]:
total_vocab_size = len(DF)
print(total_vocab_size)

144


# Exercise 5: Test the doc_freq( )for a given word!

In [16]:
def doc_freq(word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c
word ="know"
print("(word) : ",word," (Frequency) : ",doc_freq(word))

(word) :  know  (Frequency) :  3


In [18]:
doc = 0
tf_idf = {}

for i in range(N):
    
    tokens = processed_text[i]
    
    counter = Counter(tokens + processed_text[i])
    words_count = len(tokens + processed_text[i])
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1))
        
        tf_idf[doc, token] = tf*idf
        
    doc += 1

In [21]:
tf_idf

{(0, 'also'): 0.010868630292893419,
 (0, 'anyon'): 0.010868630292893419,
 (0, 'around'): 0.01949554748668415,
 (0, 'call'): 0.01949554748668415,
 (0, 'code'): 0.01949554748668415,
 (0, 'could'): 0.010868630292893419,
 (0, 'creat'): 0.010868630292893419,
 (0, 'describ'): 0.01949554748668415,
 (0, 'ersysedmontonabca'): 0.01949554748668415,
 (0, 'file'): 0.01949554748668415,
 (0, 'ftp'): 0.010868630292893419,
 (0, 'got'): 0.01949554748668415,
 (0, 'ie'): 0.01949554748668415,
 (0, 'imag'): 0.010868630292893419,
 (0, 'joe'): 0.0389910949733683,
 (0, 'joth'): 0.01949554748668415,
 (0, 'librari'): 0.021737260585786837,
 (0, 'like'): 0.010868630292893419,
 (0, 'make'): 0.01949554748668415,
 (0, 'never'): 0.01949554748668415,
 (0, 'newest'): 0.01949554748668415,
 (0, 'one'): 0.010868630292893419,
 (0, 'polygon'): 0.01949554748668415,
 (0, 'processor'): 0.01949554748668415,
 (0, 'program'): 0.01949554748668415,
 (0, 'recent'): 0.01949554748668415,
 (0, 'render'): 0.021737260585786837,
 (0, 'rend

# Exercise 6. What is the TF-IDF score for the first word in the collection?

In [22]:
tf_idf[(0, 'also')]

0.010868630292893419

In [24]:
tf_idf[(0, 'recent')]

0.01949554748668415

In [23]:
def matching_score(k, query):
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))
    
    print("Matching Score")
    print("\nQuery:", query)
    print("")
    print(tokens)
    
    query_weights = {}
    for key in tf_idf:
        
        if key[1] in tokens:
            try:
                query_weights[key[0]] += tf_idf[key]
            except:
                query_weights[key[0]] = tf_idf[key]
                
    query_weights = sorted(query_weights.items(),
                           key=lambda x: x[1],
                           reverse=True)
    
    print("")
    
    l = []
    
    for i in query_weights[:k]:
        l.append(i[0])
        
    print(l)
    
matching_score(2,"I recently got a file describing a library")

No Header
Matching Score

Query: I recently got a file describing a library

['recent', 'got', 'file', 'describ', 'librari']

[0, 1]


# Home Task
# Exercise 7. Use the whole files found in folder “comp.graphics” and compute TF-IDF for each word appears in the collection.

In [25]:
title = "used_files"
os.chdir("C:/mini_newsgroups/comp.graphics/")
paths= []
for (dirpath, dirnames, filenames) in os.walk(str(os.getcwd())+'/'+title+'/'): 
    for i in filenames: 
        paths.append(str(dirpath)+str("\\")+i)
        
print(dirpath)
paths[0]
print(len(paths))

C:\mini_newsgroups\comp.graphics/used_files/
100


In [26]:
def print_doc(id):
    file = open(dirpath+'/'+ filenames[id],'r', encoding='cp1250')
    text = file.read().strip()
    file.close()
    print(text)
    
print_doc(0)
print_doc(1)

Path: cantaloupe.srv.cs.cmu.edu!crabapple.srv.cs.cmu.edu!fs7.ece.cmu.edu!europa.eng.gtefsd.com!gatech!asuvax!cs.utexas.edu!zaphod.mps.ohio-state.edu!saimiri.primate.wisc.edu!usenet.coe.montana.edu!news.u.washington.edu!uw-beaver!cs.ubc.ca!unixg.ubc.ca!kakwa.ucs.ualberta.ca!ersys!joth
From: joth@ersys.edmonton.ab.ca (Joe Tham)
Newsgroups: comp.graphics
Subject: Where can I find SIPP?
Message-ID: <yFXJ2B2w165w@ersys.edmonton.ab.ca>
Date: Mon, 05 Apr 93 14:58:21 MDT
Organization: Edmonton Remote Systems #2, Edmonton, AB, Canada
Lines: 11

        I recently got a file describing a library of rendering routines 
called SIPP (SImple Polygon Processor).  Could anyone tell me where I can 
FTP the source code and which is the newest version around?
        Also, I've never used Renderman so I was wondering if Renderman 
is like SIPP?  ie. a library of rendering routines which one uses to make 
a program that creates the image...

                                        Thanks,  Joe Tham

--
Jo

In [27]:
processed_text = []

for i in range(len(filenames)):
    file = open(dirpath+'/'+ filenames[i], 'r', encoding='cp1250',errors='ignore')
    text = file.read().strip()
    file.close()
    
    processed_text.append(word_tokenize(str(preprocess(text))))
    print(processed_text)

[['recent', 'got', 'file', 'describ', 'librari', 'render', 'routin', 'call', 'sipp', 'simpl', 'polygon', 'processor', 'could', 'anyon', 'tell', 'ftp', 'sourc', 'code', 'newest', 'version', 'around', 'also', 'never', 'use', 'renderman', 'wa', 'wonder', 'renderman', 'like', 'sipp', 'ie', 'librari', 'render', 'routin', 'one', 'use', 'make', 'program', 'creat', 'imag', 'thank', 'joe', 'tham', 'joe', 'tham', 'joth', 'ersysedmontonabca']]
[['recent', 'got', 'file', 'describ', 'librari', 'render', 'routin', 'call', 'sipp', 'simpl', 'polygon', 'processor', 'could', 'anyon', 'tell', 'ftp', 'sourc', 'code', 'newest', 'version', 'around', 'also', 'never', 'use', 'renderman', 'wa', 'wonder', 'renderman', 'like', 'sipp', 'ie', 'librari', 'render', 'routin', 'one', 'use', 'make', 'program', 'creat', 'imag', 'thank', 'joe', 'tham', 'joe', 'tham', 'joth', 'ersysedmontonabca'], ['hello', 'everybodi', 'use', 'pixar', 'renderman', 'three', 'scene', 'descript', 'languag', 'creat', 'three', 'world', 'pleas

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [28]:
DF = {}
N= len(processed_text)

for i in range(N):
    tokens = processed_text[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w]= {i}
            
for i in DF:
    DF[i] = len(DF[i])

In [29]:
DF

{'recent': 7,
 'got': 12,
 'file': 25,
 'describ': 6,
 'librari': 14,
 'render': 12,
 'routin': 9,
 'call': 9,
 'sipp': 1,
 'simpl': 7,
 'polygon': 9,
 'processor': 2,
 'could': 27,
 'anyon': 30,
 'tell': 7,
 'ftp': 20,
 'sourc': 12,
 'code': 16,
 'newest': 2,
 'version': 17,
 'around': 9,
 'also': 24,
 'never': 5,
 'use': 42,
 'renderman': 2,
 'wa': 24,
 'wonder': 5,
 'like': 27,
 'ie': 11,
 'one': 75,
 'make': 16,
 'program': 29,
 'creat': 16,
 'imag': 35,
 'thank': 33,
 'joe': 4,
 'tham': 1,
 'joth': 1,
 'ersysedmontonabca': 2,
 'hello': 5,
 'everybodi': 1,
 'pixar': 2,
 'three': 61,
 'scene': 3,
 'descript': 7,
 'languag': 6,
 'world': 5,
 'pleas': 24,
 'help': 30,
 'next': 5,
 'document': 8,
 'nextstep': 2,
 'avail': 18,
 'veri': 21,
 'complic': 8,
 'surfac': 9,
 'shader': 1,
 'bring': 4,
 'life': 6,
 'appli': 6,
 'shadow': 1,
 'reflect': 1,
 'far': 7,
 'understand': 7,
 'defin': 6,
 'environment': 2,
 'map': 10,
 'produc': 5,
 'know': 38,
 'ani': 48,
 'advis': 2,
 'rib': 1,
 'exa

In [30]:
total_vocab =[x for x in DF]
print(total_vocab[:5])

['recent', 'got', 'file', 'describ', 'librari']


In [31]:
total_vocab_size = len(DF)
print(total_vocab_size)

5237


In [32]:
def doc_freq(word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c
word ="know"
print("(word) : ",word," (Frequency) : ",doc_freq(word))

(word) :  know  (Frequency) :  38


In [33]:
doc = 0
tf_idf = {}

for i in range(N):
    
    tokens = processed_text[i]
    
    counter = Counter(tokens + processed_text[i])
    words_count = len(tokens + processed_text[i])
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1))
        
        tf_idf[doc, token] = tf*idf
        
    doc += 1

In [34]:
tf_idf

{(0, 'also'): 0.029707333871767206,
 (0, 'anyon'): 0.025130496007576875,
 (0, 'around'): 0.049202881358451356,
 (0, 'call'): 0.049202881358451356,
 (0, 'code'): 0.037912918569894545,
 (0, 'could'): 0.02729608524821395,
 (0, 'creat'): 0.037912918569894545,
 (0, 'describ'): 0.05679170995289247,
 (0, 'ersysedmontonabca'): 0.07481932400368403,
 (0, 'file'): 0.02887285061318675,
 (0, 'ftp'): 0.03341698040676248,
 (0, 'got'): 0.04362066296552601,
 (0, 'ie'): 0.04532369929900551,
 (0, 'imag'): 0.021948969752875518,
 (0, 'joe'): 0.12790138742158125,
 (0, 'joth'): 0.08344624119747476,
 (0, 'librari'): 0.08115192832932125,
 (0, 'like'): 0.02729608524821395,
 (0, 'make'): 0.037912918569894545,
 (0, 'never'): 0.06007151165134477,
 (0, 'newest'): 0.07481932400368403,
 (0, 'one'): 0.0060507909905303916,
 (0, 'polygon'): 0.049202881358451356,
 (0, 'processor'): 0.07481932400368403,
 (0, 'program'): 0.02582815181232136,
 (0, 'recent'): 0.05395061649279624,
 (0, 'render'): 0.08724132593105202,
 (0, 're

In [35]:
tf_idf[(0, 'also')]

0.029707333871767206

In [36]:
def matching_score(k, query):
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))
    
    print("Matching Score")
    print("\nQuery:", query)
    print("")
    print(tokens)
    
    query_weights = {}
    for key in tf_idf:
        
        if key[1] in tokens:
            try:
                query_weights[key[0]] += tf_idf[key]
            except:
                query_weights[key[0]] = tf_idf[key]
                
    query_weights = sorted(query_weights.items(),
                           key=lambda x: x[1],
                           reverse=True)
    
    print("")
    
    l = []
    
    for i in query_weights[:k]:
        l.append(i[0])
        
    print(l)
    
matching_score(2,"I recently got a file describing a library")

No Header
Matching Score

Query: I recently got a file describing a library

['recent', 'got', 'file', 'describ', 'librari']

[0, 53]
